In [42]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_core.messages import SystemMessage, trim_messages
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.llms import Ollama
from langchain_ollama import ChatOllama
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [2]:
# load_dotenv()
# 
# groq_api_key= os.getenv("GROQ_API_KEY")

## Building A Chatbot

In [23]:
#llm=ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
llm = ChatOllama(model="llama3.2:latest")

In [24]:
llm.invoke([HumanMessage(content="Hi! My name is Charles")])

AIMessage(content="Hello Charles! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2024-11-30T17:37:55.080509266Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1056736431, 'load_duration': 872440193, 'prompt_eval_count': 31, 'prompt_eval_duration': 48000000, 'eval_count': 26, 'eval_duration': 134000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-9bf34d16-b858-4068-b13d-082393dc74b0-0', usage_metadata={'input_tokens': 31, 'output_tokens': 26, 'total_tokens': 57})

In [5]:
messages=[
    HumanMessage(content="Hi! My name is Charles"),
    AIMessage(content="Hi Charles, it's nice to meet you! What can I do for you today?"),
    HumanMessage(content="What's my name")
]

In [6]:
llm.invoke(messages)

AIMessage(content='Your name is Charles. 😊  \n\nI remember that you told me!  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 49, 'total_tokens': 78, 'completion_time': 0.052727273, 'prompt_time': 0.000537448, 'queue_time': 0.014050682, 'total_time': 0.053264721}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-03a6ad65-53c4-43cb-a60e-5317e93c08c2-0', usage_metadata={'input_tokens': 49, 'output_tokens': 29, 'total_tokens': 78})

#### Message history

In [7]:
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(llm, get_session_history)

In [8]:
config={"configurable":{"session_id":"chat2"}}

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi! My name is Charles")],
    config=config
)

In [10]:
print(response.content)

Hi Charles, it's nice to meet you! What can I do for you today? 😄  




In [11]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content='Your name is Charles.  \n\nI remember that you told me! 😊  Is there anything else I can help you with? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 53, 'total_tokens': 83, 'completion_time': 0.054545455, 'prompt_time': 0.001818043, 'queue_time': 0.012813837, 'total_time': 0.056363498}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-334d616e-1194-4692-8944-b94843bd6f98-0', usage_metadata={'input_tokens': 53, 'output_tokens': 30, 'total_tokens': 83})

#### change the config --> session id

In [12]:
config1={"configurable":{"session_id":"chat2"}}

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)
response.content

"You said your name is Charles!  \n\nI'm ready for another question if you have one.  😊 \n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hello, my name is John.")],
    config=config1
)
response.content

'Hello John! Nice to meet you.  😊\n\nIs there anything I can help you with today?  \n'

#### Prompt Templates

In [15]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI assistant. Answer all the questions to the best of your ability. If you don't an answer then say you don't know."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|llm

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi, my name is Charles")]})

AIMessage(content="Hello Charles, it's nice to meet you! \n\nI'm ready to answer your questions. What can I do for you? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 48, 'total_tokens': 83, 'completion_time': 0.063636364, 'prompt_time': 0.000535868, 'queue_time': 0.014006722, 'total_time': 0.064172232}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-53cafb7c-a190-4b46-8c9c-5791f6fbf140-0', usage_metadata={'input_tokens': 48, 'output_tokens': 35, 'total_tokens': 83})

In [17]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [18]:
config1={"configurable":{"session_id":"chat3"}}

In [19]:
response=with_message_history.invoke(
        [HumanMessage(content="Hello, my name is Charles.")],
    config=config
)
response

AIMessage(content="Hello Charles! It's nice to hear from you again.  \n\nWhat can I do for you today?  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 212, 'total_tokens': 239, 'completion_time': 0.049090909, 'prompt_time': 0.006716045, 'queue_time': 0.006927265, 'total_time': 0.055806954}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-00656814-75a5-4836-b666-c73cdc2e0b2d-0', usage_metadata={'input_tokens': 212, 'output_tokens': 27, 'total_tokens': 239})

In [20]:
response= chain.invoke({
        "messages":[HumanMessage(content="Hello, my name is Charles.")],"language":"French"})
response.content


"Hello Charles, it's nice to meet you! \n\nI'm ready for your questions. Ask away! 😊  \n"

In [ ]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [ ]:
config={"configurable":{"session_id":"chat4"}}

In [ ]:
response=with_message_history.invoke(
   {"messages":[HumanMessage(content="Hello, my name is Charles.")],"language":"French"},
   config=config
)
response.content

In [ ]:
response=with_message_history.invoke(
   {"messages":[HumanMessage(content="What's my name?.")],"language":"English"},
   config=config
)
response.content

## Managing the Conversation History

In [39]:
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [40]:
messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="hi!, I'm Bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice create"),
    AIMessage(content="Nice!"),
    HumanMessage(content="What's 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="Yes!"),
]

In [41]:
trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice create', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's 2 + 2", additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [43]:
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | llm
)

In [47]:
response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What math problem did i ask?")],
    "language": "English"
}
)

In [48]:
response.content

'You asked me "2 + 2" earlier.'

In [49]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [50]:
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  messages: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x77be0f41f240>, input_messages_key='messages', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])